In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import yaml
from utils.data_processing import df_rebase, butter_lowpass_filter, split_windows

# Load configuration
config_path = os.path.join(os.getcwd(), "..", "config.yml")
with open(config_path) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

# Load single instance data
single_instance_path = os.path.join(os.getcwd(), "..", config["single_instance_path"])
df = pd.read_csv(single_instance_path)
df = df_rebase(df, config['order'], config['rename'])

# Plot the time-length of the instance
time_length = df.shape[0]
plt.bar(["Instance"], [time_length])
plt.ylabel('Time-length (samples)')
plt.title('Time-length of Single Instance')
plt.show()

# Split data into fixed windows
window_size = config["sliding_window"]["ws"]
overlap = config["sliding_window"]["overlap"]
windows = split_windows(df, window_size, overlap)

# Filter the data
cutoff = config["filter"]["wn"]
fs = 50  # Sample rate
order = config["filter"]["order"]

df["acc_x"] = butter_lowpass_filter(df["acc_x"], cutoff, fs, order)
df["acc_y"] = butter_lowpass_filter(df["acc_y"], cutoff, fs, order)
df["acc_z"] = butter_lowpass_filter(df["acc_z"], cutoff, fs, order)

# Transform data into frequency domain
df_fft = pd.DataFrame(np.abs(np.fft.fft(df, axis=0)), columns=df.columns)

# Visualize time-series instance after filtering
plt.figure(figsize=(12, 6))
plt.plot(df["acc_x"], label='acc_x')
plt.plot(df["acc_y"], label='acc_y')
plt.plot(df["acc_z"], label='acc_z')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.title('Filtered Signal')
plt.legend()
plt.show()
